In [19]:
import pandas as pd
BN_BNA_sup_5_2022 = pd.read_csv("C:/Users/Lenovo/Documents/WCS/Mission_DATA/CSV/BN_BNA_sup_5_2022.csv", sep=",")
tour1_2022 = pd.read_csv("C:/Users/Lenovo/Documents/WCS/Mission_DATA/CSV/meilleurs_candidats_tour1_22.csv", sep=",")


In [20]:
BN_BNA_sup_5_2022['Libellé de la commune'].nunique()

31236

In [21]:
BN_BNA_sup_5_2022.columns

Index(['Unnamed: 0', 'Tour', 'Année_élection', 'Libellé de la commune',
       'Blancs et nuls', 'Blancs_nuls et abstentions', 'Coordonnées',
       'longitude', 'latitude'],
      dtype='object')

In [23]:
tour1_2022.columns

Index(['Unnamed: 0', 'index', 'Départements', 'Candidats', 'Votes',
       'coordonnées', 'longitude', 'latitude'],
      dtype='object')

In [25]:
tour1_2022['Départements'].nunique()

87

In [26]:
tour1_2022['Candidats'].value_counts()

Candidats
Blancs_nuls et abstentions    77
Emmanuel MACRON                7
Marine LE PEN                  6
Jean-Luc MÉLENCHON             1
Name: count, dtype: int64

In [ ]:
import folium
from folium.plugins import MarkerCluster

# supprimer les nan et changer le format de la colonne Tour en string
sup5_BN_BNA_sans_nan = BN_BNA_sup_5_2022.dropna()
sup5_BN_BNA_sans_nan['Tour'] = sup5_BN_BNA_sans_nan['Tour'].astype(str)

tours = list(sup5_BN_BNA_sans_nan['Tour'].unique()) # faire afficher les valeurs uniques de la colonne

# centrer la carte sur la France
latitude_centre = 46.603354
longitude_centre = 1.888152
m = folium.Map(location=[latitude_centre, longitude_centre], zoom_start=6)

# créer des groupes pour chaque tour
groupes_tours = {}

for tour in tours:
    filtre = sup5_BN_BNA_sans_nan[sup5_BN_BNA_sans_nan['Tour'] == tour] # créer un filtre pour chaque tour
    groupe = folium.FeatureGroup(name=f"Tour {tour}")  # créée des groupes filtrables en utilisant FeatureGroup qui permet d’activer ou désactiver les marqueurs
    marker_cluster = MarkerCluster().add_to(groupe)

    for _, row in filtre.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            tooltip=f"<b>Commune:</b> {row['Libellé de la commune']}<br><b>Blancs et nuls:</b> {row['Blancs et nuls']}<br><b>Blancs, nuls et abstentions:</b> {row['Blancs_nuls et abstentions']}",
            icon=folium.Icon(color='purple')
        ).add_to(marker_cluster)

    groupe.add_to(m)  # Ajout du groupe à la carte
    groupes_tours[tour] = groupe  # Stockage du groupe

# Ajout du contrôle des couches
folium.LayerControl(collapsed=False).add_to(m) # LayerControl permet à l’utilisateur d’afficher les tours avec des cases à cocher

# Affichage de la carte
m.save("map_sup5.html")
m


In [ ]:
import folium
from folium.plugins import MarkerCluster

tour1_2022_sans_nan = tour1_2022.dropna()

# centrer la carte sur la France
latitude_centre = 46.603354
longitude_centre = 1.888152
m = folium.Map(location=[latitude_centre, longitude_centre], zoom_start=6)

# créer un dictionnaire pour stocker les groupes par candidat
groupes_candidats = {}

# parcours la liste des candidats afin de créer un groupe pour chacun
for candidat in tour1_2022_sans_nan["Candidats"].unique():
    groupe = folium.FeatureGroup(name=candidat)  # créee le groupe pour un candidat
    marker_cluster = MarkerCluster().add_to(groupe)  # ajoute du clustering

    # filtrer les données selon le candidat
    df_filtre = tour1_2022_sans_nan[tour1_2022_sans_nan["Candidats"] == candidat]

    # code pour ajouter des marqueurs avec texte
    for _, row in df_filtre.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            tooltip=f"<b>{candidat}<br><b>Département:</b> {row['Départements']}<br><b>Score:</b> {row['Votes']}",
            icon=folium.Icon(color="blue")
        ).add_to(marker_cluster)

    # Ajout du groupe à la carte
    groupes_candidats[candidat] = groupe
    m.add_child(groupe)

# contrôle des groupes sur la carte
folium.LayerControl().add_to(m)

# Affichage de la carte
m.save("map_departements.html")
m

